# Volcano Data Acquisition with Google Earth Engine

This notebook demonstrates how to download Digital Elevation Models (DEMs) and Land Cover data for volcanic regions using Google Earth Engine. These datasets are essential for creating cost surfaces and analyzing evacuation routes.

## Why is this data important?

For volcanic evacuation analysis, we need:

1. **Digital Elevation Model (DEM)** - Provides terrain elevation data used to calculate slopes and identify evacuation challenges
2. **Land Cover Classification** - Identifies different types of terrain (forests, fields, water bodies, etc.) that affect travel speed
3. **Geographic Context** - Buffers around volcanoes provide the necessary spatial extent for comprehensive evacuation planning

In this notebook, we'll download this data for Mount Marapi and Mount Awu in Indonesia, but the process can be applied to any volcano worldwide.

## Importing Required Packages

First, we need to import the necessary library:

- **earthengine-api**: Provides access to Google Earth Engine

In [19]:
# Import required packages
import ee


## Google Earth Engine Authentication

Before accessing any data, we need to authenticate with Google Earth Engine. This function will:

1. Check if you're already authenticated
2. If not, it will open a browser window for you to sign in with your Google account
3. Authorize the Earth Engine application



In [20]:
def authenticate_gee():
    """
    Authenticate with Google Earth Engine.
    
    This function checks if you're already authenticated and if not,
    initiates the authentication process.
    """
    try:
        # Try to initialize without authentication
        ee.Initialize()
        print("Already authenticated with Google Earth Engine")
    except:
        # If initialization fails, authenticate first
        ee.Authenticate()
        ee.Initialize()
        print("Authentication successful")

# Run the authentication function
authenticate_gee()

Already authenticated with Google Earth Engine


## Determining the Correct UTM Projection

When working with geospatial data for specific locations, using the correct Universal Transverse Mercator (UTM) projection is crucial for accurate distance and area measurements. Rather than manually specifying the EPSG code, we can calculate it automatically based on the volcano's coordinates.

The function below:
1. Takes latitude and longitude as input
2. Calculates the appropriate UTM zone number (1-60)
3. Determines whether the location is in the northern or southern hemisphere
4. Returns the correct EPSG code for that UTM zone

This ensures that our data will always be processed in the most appropriate projection for the volcano's location.

In [21]:
def get_utm_epsg(latitude, longitude):
    """
    Determine the EPSG code for the UTM zone appropriate for the given coordinates.
    
    Args:
        latitude (float): Latitude in decimal degrees
        longitude (float): Longitude in decimal degrees
        
    Returns:
        str: EPSG code in the format "EPSG:xxxxx"
    """
    # Make sure longitude is between -180 and 180
    longitude = ((longitude + 180) % 360) - 180
    
    # Calculate UTM zone number
    zone_number = int(((longitude + 180) / 6) + 1)
    
    # Special cases for Norway and Svalbard
    if 56 <= latitude < 64 and 3 <= longitude < 12:
        zone_number = 32
    elif 72 <= latitude < 84 and longitude >= 0:
        if longitude < 9:
            zone_number = 31
        elif longitude < 21:
            zone_number = 33
        elif longitude < 33:
            zone_number = 35
        elif longitude < 42:
            zone_number = 37
    
    # Determine EPSG code
    if latitude >= 0:
        # Northern hemisphere
        epsg = f"EPSG:326{zone_number:02d}"
    else:
        # Southern hemisphere
        epsg = f"EPSG:327{zone_number:02d}"
    
    print(f"Coordinates ({latitude}, {longitude}) are in UTM zone {zone_number}")
    print(f"Using projection {epsg}")
    
    return epsg

## Defining the Area of Interest (AOI)

For each volcano, we need to define an area of interest by:

1. Creating a point at the volcano's coordinates (longitude, latitude)
2. Creating a buffer around this point 
3. Extracting the region coordinates for export boundaries



This function returns both the Earth Engine geometry object and the region coordinates.

In [22]:
def define_aoi(coords, buffer_distance=20000):
    """
    Define an area of interest around a point.
    
    Args:
        coords (list): [longitude, latitude] of the point
        buffer_distance (int): Buffer distance in meters
        
    Returns:
        tuple: (ee.Geometry, list) - AOI and region coordinates
    """
    # Create a point from coordinates
    point = ee.Geometry.Point(coords)
    
    # Create a buffer around the point
    aoi = point.buffer(buffer_distance)
    
    # Get the region coordinates
    region_coords = aoi.bounds().getInfo()['coordinates']
    
    print(f"Created AOI with {buffer_distance/1000} km buffer around {coords}")
    
    return aoi, region_coords

## Downloading Digital Elevation Model (DEM) Data

Now we'll create a function to download DEM data. We're using the ALOS World 3D-30m dataset from JAXA, which provides global elevation data at approximately 30-meter resolution.

This function:
1. Loads the DEM collection from Google Earth Engine
2. Creates a mosaic of all available images
3. Clips it to our area of interest
4. Reprojects to a UTM projection appropriate for the location
5. Exports the result to Google Drive as a GeoTIFF file



The function returns an export task that runs asynchronously in the cloud.

In [23]:
def download_dem(aoi, region_coords, scale, description, utm_epsg, collection="JAXA/ALOS/AW3D30/V3_2", band="DSM"):
    """
    Download DEM data for the specified area.
    
    Args:
        aoi (ee.Geometry): Area of interest
        region_coords (list): Region coordinates
        scale (int): Resolution in meters
        description (str): Description for the export task
        utm_epsg (str): UTM projection code
        collection (str): GEE collection ID
        band (str): Band to select
        
    Returns:
        ee.batch.Task: Export task
    """
    # Load DEM collection
    dem_collection = ee.ImageCollection(collection).select(band)
    
    # Create a mosaic (combines all images in the collection)
    dem_mosaic = dem_collection.mosaic().clip(aoi)
    
    # Reproject to UTM with specified scale
    utm_projection = ee.Projection(utm_epsg).atScale(scale)
    dem_utm = dem_mosaic.reproject(utm_projection)
    
    # Export to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=dem_utm,
        description=f"{description}_{scale}m_Buffer_UTM",
        scale=scale,
        region=region_coords,
        crs=utm_epsg,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    
    # Start the task
    task.start()
    
    print(f"DEM export started: {description}_{scale}m_Buffer_UTM.tif")
    print(f"This file will be available in your Google Drive when processing completes")
    
    return task

## Downloading Land Cover Data

Next, we'll create a function to download land cover data. We're using the Copernicus Global Land Cover dataset, which provides global land cover classification at 100-meter resolution.

The land cover dataset classifies each pixel into categories such as:
- Forests (different types)
- Shrublands
- Grasslands
- Croplands
- Urban areas
- Water bodies


Like the DEM function, this function exports the data to Google Drive as a GeoTIFF file.

In [24]:
def download_landcover(aoi, region_coords, year, scale, description, utm_epsg):
    """
    Download land cover data for the specified area.
    
    Args:
        aoi (ee.Geometry): Area of interest
        region_coords (list): Region coordinates
        year (int): Year of land cover data
        scale (int): Resolution in meters
        description (str): Description for the export task
        utm_epsg (str): UTM projection code
        
    Returns:
        ee.batch.Task: Export task
    """
    # Load land cover dataset for the specified year
    dataset = ee.Image(f'COPERNICUS/Landcover/100m/Proba-V-C3/Global/{year}').select('discrete_classification')
    
    # Clip to AOI
    clipped_dataset = dataset.clip(aoi)
    
    # Reproject to UTM
    utm_projection = ee.Projection(utm_epsg).atScale(scale)
    dataset_utm = clipped_dataset.reproject(utm_projection)
    
    # Export to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=dataset_utm,
        description=f"{description}_{year}_{scale}m_Buffer_UTM",
        scale=scale,
        region=region_coords,
        crs=utm_epsg,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    
    # Start the task
    task.start()
    
    print(f"Land cover export started: {description}_{year}_{scale}m_Buffer_UTM.tif")
    print(f"This file will be available in your Google Drive when processing completes")
    
    return task

## Main Function for Downloading Volcano Data

Now let's create a main function that combines all the previous functions to download both DEM and land cover data for a specified volcano. This function:

1. Authenticates with Google Earth Engine
2. Defines the area of interest around the volcano
3. Downloads the DEM data
4. Downloads the land cover data
5. Returns the export tasks for status monitoring

This modular approach makes it easy to download data for multiple volcanoes with different parameters.

### Parameters:

- **volcano_name**: Name of the volcano (used in output filenames)
- **coords**: [longitude, latitude] coordinates of the volcano
- **buffer_distance**: Buffer distance in meters (default: 20000 m, or 20 km)
- **scale**: Resolution in meters (default: 100 m)
- **year**: Year of land cover data (default: 2019)
- **utm_epsg**: UTM projection code appropriate for the volcano's location

In [25]:
def download_volcano_data(volcano_name, lat, lon, buffer_distance=20000, scale=100, year=2019, utm_epsg=None):
    """
    Download DEM and land cover data for a volcano with automatic UTM projection detection.
    
    Args:
        volcano_name (str): Name of the volcano
        lat (float): Latitude of the volcano summit
        lon (float): Longitude of the volcano summit
        buffer_distance (int): Buffer distance in meters
        scale (int): Resolution in meters
        year (int): Year of land cover data
        utm_epsg (str, optional): UTM projection code. If None, it will be determined automatically.
        
    Returns:
        tuple: (ee.batch.Task, ee.batch.Task) - DEM and land cover export tasks
    """
    # Get the proper UTM EPSG if not provided
    if utm_epsg is None:
        utm_epsg = get_utm_epsg(lat, lon)
    
    # Define area of interest
    coords = [lon, lat]  # GEE uses [longitude, latitude] order
    aoi, region_coords = define_aoi(coords, buffer_distance)
    
    # Download DEM
    dem_task = download_dem(
        aoi, 
        region_coords, 
        scale, 
        f"{volcano_name}_DEM", 
        utm_epsg
    )
    
    # Download land cover
    lc_task = download_landcover(
        aoi, 
        region_coords, 
        year, 
        scale, 
        f"{volcano_name}_LandCover", 
        utm_epsg
    )
    
    print(f"Data export initiated for {volcano_name}")
    print(f"Using {utm_epsg} projection (automatically determined)")
    print(f"Buffer: {buffer_distance/1000} km, Resolution: {scale} m")
    print(f"The processing will continue in the cloud even if you close this notebook")
    
    return dem_task, lc_task

## Download Data for Mount Marapi

Let's use our function to download data for Mount Marapi in Sumatra, Indonesia. Mount Marapi is an active stratovolcano and one of the most active volcanoes in Sumatra.

- **Location**: 100.4721°E, -0.3775°N


For Mount Marapi, we'll:
1. We just provide the name and coordinates
2. We use a 20 km buffer and 100 m resolution 


In [26]:
# Mount Marapi coordinates in Sumatra
MARAPI_LAT = -0.3775
MARAPI_LON = 100.4721

# Download data for Mount Marapi with automatic UTM detection
marapi_tasks = download_volcano_data(
    volcano_name="Marapi", 
    lat=MARAPI_LAT,
    lon=MARAPI_LON,
    buffer_distance=20000, 
    scale=100
)

Coordinates (-0.3775, 100.47210000000001) are in UTM zone 47
Using projection EPSG:32747
Created AOI with 20.0 km buffer around [100.4721, -0.3775]
DEM export started: Marapi_DEM_100m_Buffer_UTM.tif
This file will be available in your Google Drive when processing completes
Land cover export started: Marapi_LandCover_2019_100m_Buffer_UTM.tif
This file will be available in your Google Drive when processing completes
Data export initiated for Marapi
Using EPSG:32747 projection (automatically determined)
Buffer: 20.0 km, Resolution: 100 m
The processing will continue in the cloud even if you close this notebook


## Download Data for Mount Awu

Now let's download data for Mount Awu in the Sangihe Islands, Indonesia. Mount Awu is a stratovolcano with a history of explosive eruptions.

- **Location**: 125.4542°E, 3.6735°N


For Mount Awu, we'll:
1. Use a 20 km buffer 
2. Set the resolution to 100 meters (as with Marapi)


In [ ]:
# Mount Awu coordinates in Sangihe Islands
AWU_LAT = 3.6828460
AWU_LON = 125.455980

# Download data for Mount Awu with automatic UTM detection
awu_tasks = download_volcano_data(
    volcano_name="Awu", 
    lat=AWU_LAT,
    lon=AWU_LON,
    buffer_distance=20000,
    scale=100
)


Coordinates (3.682846, 125.45598000000001) are in UTM zone 51
Using projection EPSG:32651
Created AOI with 15.0 km buffer around [125.45598, 3.682846]
DEM export started: Awu_DEM_100m_Buffer_UTM.tif
This file will be available in your Google Drive when processing completes
Land cover export started: Awu_LandCover_2019_100m_Buffer_UTM.tif
This file will be available in your Google Drive when processing completes
Data export initiated for Awu
Using EPSG:32651 projection (automatically determined)
Buffer: 15.0 km, Resolution: 100 m
The processing will continue in the cloud even if you close this notebook


## After Downloading: Next Steps

After your export tasks complete, here's what to do next:

1. **Download the files from Google Drive**:
   - Navigate to your Google Drive
   - Look for files named `Marapi_DEM_100m_Buffer_UTM.tif` and `Marapi_LandCover_2019_100m_Buffer_UTM.tif` (similarly for Awu)
   - Download these files to your local project directory

2. **Verify the data**:
   - Open the files in a GIS software like QGIS or ArcGIS
   - Check that the data covers the expected area
   - Verify that there are no major gaps or errors

3. **Proceed to the next step**:
- Continue to the [Cost Surface Generation](cost-surface.ipynb) notebook
- There you'll use these datasets to create cost surfaces for evacuation analysis

## Conclusion

In this notebook, we've:

1. **Set up Google Earth Engine** for geospatial data access
2. **Downloaded DEM data** from the ALOS World 3D-30m dataset
3. **Downloaded land cover data** from the Copernicus Global Land Cover dataset
4. **Created a reusable module** for downloading data for any volcano

These datasets will form the foundation for our volcano evacuation analysis. In the next notebook, we'll use this data to:

1. Calculate slopes from the DEM
2. Assign travel speeds to different land cover types
3. Combine these factors to create cost surfaces
4. Use these cost surfaces for evacuation routing

